In [ ]:
import shutil, os
if os.path.isdir('rainbow') : shutil.rmtree("rainbow")
!git clone https://github.com/ClementPerroud/Rainbow-Agent rainbow

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from sklearn.preprocessing import robust_scale

from rainbow.agent import Rainbow
from preprocess import preprocess

import sys
import gym_trading_env
import nest_asyncio

/Users/manager/source/sb3/.venv/lib/python3.11/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# Create environments

In [2]:
def add_features(df):
    df["feature_close"] = robust_scale(df["close"].pct_change())
    df["feature_open"] = robust_scale(df["open"]/df["close"])
    df["feature_high"] = robust_scale(df["high"]/df["close"])
    df["feature_low"] = robust_scale(df["low"]/df["close"])
    df["feature_volume"] = robust_scale(df["volume"] / df["volume"].rolling(7*24).max())
    df.dropna(inplace= True)
    return df


def reward_function(history):
    return np.log(history["portfolio_valuation", -1] / history["portfolio_valuation", -2])

def max_drawdown(history):
    networth_array = history['portfolio_valuation']
    _max_networth = networth_array[0]
    _max_drawdown = 0
    for networth in networth_array:
        if networth > _max_networth:
            _max_networth = networth
        drawdown = ( networth - _max_networth ) / _max_networth
        if drawdown < _max_drawdown:
            _max_drawdown = drawdown
    return f"{_max_drawdown*100:5.2f}%"

def make_env(dir):
    env = gym.make(
        "MultiDatasetTradingEnv",
        dataset_dir= dir,
        preprocess= preprocess,
        windows= 15,
        positions = [ -1, -0.5, 0, 1, 2],
        initial_position = 0,
        trading_fees = 0.01/100,
        borrow_interest_rate= 0.0003/100,
        reward_function = reward_function,
        portfolio_initial_value = 1000,
        verbose=1,
    )
    env.unwrapped.add_metric('Position Changes', lambda history : f"{ 100*np.sum(np.diff(history['position']) != 0)/len(history['position']):5.2f}%" )
    env.unwrapped.add_metric('Max Drawdown', max_drawdown)
    return env

training_envs = gym.vector.SyncVectorEnv([lambda: make_env("./data/train/month/**/*.pkl") for _ in range(5)])
validation_envs = gym.vector.SyncVectorEnv([lambda: make_env("./data/test/month/2023/*.pkl") for _ in range(5)])


In [3]:
agent = Rainbow(
    simultaneous_training_env = 5,
    
    #Distributional
    distributional= True,
    v_min= -200,
    v_max = 250,
    nb_atoms= 51, 
    # Prioritized Replay
    prioritized_replay = False,
    prioritized_replay_alpha= 0.5,
    prioritized_replay_beta_function = lambda episode, step : min(1, 0.5 + 0.5*step/150_000),
    
    # General
    multi_steps = 3,
    nb_states = 14,
    nb_actions = 4,
    gamma = 0.99,
    replay_capacity = 1E8,
    tau = 2000,
    
    # Model
    window= 15,
    units = [16,16, 16],
    dropout= 0.2,
    adversarial= True,
    noisy= False,
    learning_rate = 3*2.5E-4,

    batch_size= 128,
    train_every = 10,
    epsilon_function = lambda episode, step : max(0.001, (1 - 5E-5)** step),
    name = "Rainbow",
)

In [4]:
def train(steps = 100_000):
    print("___________________________________________ TRAINING ___________________________________________")
    if 'obs' not in globals():
        global obs
        obs, info = training_envs.reset()
    for _ in range(steps):
        actions = agent.e_greedy_pick_actions(obs)
        next_obs, rewards, dones, truncateds, infos = training_envs.step(actions)

        agent.store_replays(obs, actions, rewards, next_obs, dones, truncateds)
        agent.train()

        obs = next_obs

def evaluation():
    print("___________________________________________ VALIDATION ___________________________________________")
    val_obs, info = validation_envs.reset()
    check = np.array([False for _ in range(val_obs.shape[0])])
    while not np.all(check):
        actions = agent.e_greedy_pick_actions(val_obs)
        next_obs, rewards, dones, truncateds, infos = validation_envs.step(actions)
        val_obs = next_obs
        check += dones + truncateds

In [5]:
while True:
    train(steps = 30_000)
    evaluation()

___________________________________________ TRAINING ___________________________________________
___________________________________________ VALIDATION ___________________________________________
Market Return :  1.34%   |   Portfolio Return : -82.22%   |   Position Changes : 29.76%   |   Max Drawdown : -82.33%   |   
Market Return : -9.37%   |   Portfolio Return : -85.61%   |   Position Changes : 29.37%   |   Max Drawdown : -85.75%   |   
Market Return : -9.37%   |   Portfolio Return : -84.68%   |   Position Changes : 29.45%   |   Max Drawdown : -84.84%   |   
Market Return : -2.43%   |   Portfolio Return : -85.19%   |   Position Changes : 29.67%   |   Max Drawdown : -85.35%   |   
Market Return : -7.14%   |   Portfolio Return : -85.30%   |   Position Changes : 29.34%   |   Max Drawdown : -85.39%   |   
___________________________________________ TRAINING ___________________________________________
Market Return : 36.20%   |   Portfolio Return : -88.47%   |   Position Changes : 46.33%

In [ ]:
import dill, pickle
#agent.model = None
#agent.target_model = None
#agent.replay_memory = None

with open("test.pkl", "wb") as file:
    dill.dump(agent, file)

In [ ]:
batch_indexes, states, actions, rewards, states_prime, dones, importance_weights = agent.replay_memory.sample(
    256,
    agent.prioritized_replay_beta_function(agent.episode_count, agent.steps)
)
results = agent.model(states)

action_colors=["blue", "orange","purple","red"]
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize=(16,9), dpi=300)
for action in range(4):
    for i in range(256):
        axes[action%2, action//2%2].plot(agent.zs, results[i, action, :], color = action_colors[action], alpha = 0.2)

In [ ]:
batch_indexes, states, actions, rewards, states_prime, dones, importance_weights = agent.replay_memory.sample(
    256,
    agent.prioritized_replay_beta_function(agent.episode_count, agent.steps)
)
results = agent.model(states)

action_colors=["blue", "orange","purple","red"]
fig, axes = plt.subplots(nrows = 2, ncols = 2, figsize=(16,9), dpi=300)
for action in range(4):
    for i in range(1):
        axes[action%2, action//2%2].plot(agent.zs, results[i, action, :], color = action_colors[action], alpha = 0.2)